# Libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Basic libraries
#
import random
import numpy  as np
import pandas as pd
from   pickle import dump
from   tqdm   import tqdm


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Sklearn library
#
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Visualization library
#
import matplotlib.pyplot   as plt 


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# PyTorch library
#
import torch


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# User library
#
from data_loader.data_loader  import *
from utils.PerformanceMetrics import RegressionEvaluation

In [ ]:
# Sets the seed 
#
seed = 42
random.seed( seed )
torch.manual_seed( seed )
np.random.seed( seed )

# Parameters

In [ ]:
class Parameters():
    def __init__(self):
        self.description = 'Autoformer & Transformer family for Time Series Forecasting'

        
        # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        # Basic config
        #
        #
        # Model ID
        self.model_id = 'Exchange_rate_24_24'
        # Select model, options: [Autoformer, Informer, Transformer]
        self.model    = 'Autoformer'


        # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        # Data loader
        #
        #
        # Root path of the data file
        self.root_path = './Datasets/exchange_rate/'
        # Data filename
        self.data_path = 'exchange_rate.csv'
        # Target feature in S or MS task
        self.target = 'OT'
        # Log-transformation
        self.transformation = True
        # Scaling, options: ['Standard', 'Robust', MinMax']
        self.scaling = 'Standard'
        # Forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, 
        # S:univariate predict univariate, MS: multivariate predict univariate
        self.features  = 'M'
        # Freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, 
        # b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
        self.freq      = 'h'              
        # Location of model checkpoints
        self.checkpoints = './checkpoints/'
        # Save examples during testing
        self.saveExamples = False
        
        # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        # Forecasting task
        #
        #
        # Input sequence length
        self.seq_len   = 24  
        # Start token length
        self.label_len = 12  
        # Prediction sequence length
        self.pred_len  = 24  



        # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        # Model hyper-parameters
        #
        #
        # Encoder input size
        self.enc_in     = 8
        # Decoder input size        
        self.dec_in     = 8
        # Output size
        self.c_out      = 8
        # Dimension of the model
        self.d_model    = 512
        # Number of heads
        self.n_heads    = 8
        # Number of encoder layers
        self.e_layers   = 2
        # Number of decoder layers
        self.d_layers   = 1
        # Dimension of fcn
        self.d_ff       = 2048
        # Window size of moving average
        self.moving_avg = 25
        # Attention factor
        self.factor     = 3
        # Whether to use distilling in encoder, using this argument means not using distilling
        self.distil     = True
        # Dropout rate
        self.dropout    = 0.05
        # Time features encoding, options:[timeF, fixed, learned]
        self.embed      = 'timeF'
        # Activation function
        self.activation       = 'gelu'
        # Whether to output attention in encoder
        self.output_attention = False



        # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        # Optimization
        #
        #
        # Number of workers in data-loader
        self.num_workers   = 0 # NOTE: if num_workers > 0 CUDA stops working after a while
        # Training epochs
        self.train_epochs  = 100
        # Batch size
        self.batch_size    = 32
        # Early stopping patience
        self.patience      = 10
        # Optimizer, options ['Adam', 'SGD', 'Adagrad']
        self.optimizer     = 'Adagrad'
        # Optimization learning rate
        self.learning_rate = 0.0001
        # Momentum (in case Optimizer = 'SGD')
        self.momentum = 0.9
        # Experiment description
        self.des           = 'Exp'
        # Loss function, options: [MSE, MAE, MAPE, SMAPE]
        self.loss          = 'MSE' 
        # adjust learning rate, options: ['type1', 'type2', 'Scheduler']
        self.lradj         = 'Scheduler' 
        # use automatic mixed precision training
        self.use_amp       = False

        
        # =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
        # GPU
        #
        #
        # Use gpu
        self.use_gpu       = True
        # Selected GPU
        self.gpu           = 0
        # use multi-gpu
        self.use_multi_gpu = False
        # device ids of multile gpus
        self.devices       = '0,1,2,3'   
        
    def print(self):
        d = self.__dict__
        for x in d.keys():
            print('{}: {}'.format(x, d[x]))
    
    def save(self, path = '.'):
        import json
        d = self.__dict__
        
        # create json object from dictionary
        json = json.dumps( d )
        
        
        # open file for writing, "w" 
        f = open(os.path.join(path, "Parameters.json"), "w")

        # write json object to file
        f.write(json)

        # close file
        f.close()
        
        print('[INFO] Parameters saved in file: ', os.path.join(path, "Parameters.json"))
        
args = Parameters()

In [ ]:
# setting record of experiments
#
setting = '{}_{}'.format( args.model_id, args.model )
# setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}'.format(
#     args.model_id,
#     args.model,
#     args.features,
#     args.seq_len,
#     args.label_len,
#     args.pred_len,
#     args.d_model,
#     args.n_heads,
#     args.e_layers,
#     args.d_layers,
#     args.d_ff,
#     args.factor,
#     args.embed,
#     args.distil,
#     args.des, 
#     )

# Data handling

## Import data

In [ ]:
# '''
# df_raw.columns: ['date', ...(other features), target feature]
# '''
#
df = pd.read_csv( args.root_path + args.data_path )

df.head(3)

## Preprocessing

In [ ]:
# Set Date as index
#
df['date'] = df['date'].astype('datetime64')
df.set_index('date', inplace = True)

## Split training/testing 

In [ ]:
# Set index for splitting training/testing sets 
# - 80% of data are utilized for training 
# - 20% of data are utilized for testing
train_idx = int(len(df) * 0.8)


# Create training set
#
df_train = df[:train_idx]
df_test  = df[train_idx:]

In [ ]:
# Set index for splitting training/validations sets 
# - 10% of training data are utilized for validation
valid_idx = int(len(df_train) * 0.9)


# Create training set
#
df_valid = df_train[valid_idx:]
df_train = df_train[:valid_idx]

## Visualization

In [ ]:
df_train[args.target] .plot( figsize=(20, 3) );
df_valid[args.target] .plot( );
df_test[args.target] .plot( );
#
plt.legend(['Training', 'Validation', 'Testing'], fontsize = 14, frameon = False);
plt.xlabel('Time', size = 14);
plt.xticks(size = 12);
plt.ylabel(args.target, size = 14);
plt.yticks(size = 12);

### Fix Lag

In [ ]:
df_valid = pd.concat([df_train.iloc[-args.seq_len:], df_valid])
df_test  = pd.concat([df_valid.iloc[-args.seq_len:], df_test])

## Transformation/Scaling

In [ ]:
if ( args.transformation ):
    df_train = np.log( df_train )
    df_valid = np.log( df_valid )
    df_test  = np.log( df_test  )
    
    print('[INFO] Data were transformed (Log)')

In [ ]:
if   (args.scaling == 'Standard'):
    print('[INFO] Standard scaler')
    scaler = StandardScaler()
#
elif (args.scaling == 'Robust'):
    print('[INFO] Robust scaler')
    scaler = RobustScaler()
#
elif (args.scaling == 'MinMax'):
    print('[INFO] MinMax scaler')    
    scaler = MinMaxScaler()
else:
    print('[ERROR] Invalid selection of scaler')
    print('[INFO] Standard scaler is selected')
    args.scaling = 'Standard'
    scaler = StandardScaler()


# Scaling training data
#
df_train = pd.DataFrame(data    = scaler.fit_transform( df_train ),
                        index   = df_train.index,
                        columns = df_train.columns)


# Scaling testing data
#
df_valid = pd.DataFrame(data     = scaler.transform( df_valid ),
                        index    = df_valid.index,
                        columns  = df_valid.columns)

# Scaling testing data
#
df_test = pd.DataFrame(data     = scaler.transform( df_test ),
                        index   = df_test.index,
                        columns = df_test.columns)

### Save scaler

In [ ]:
# Create directory for model checkpoints
#
path = os.path.join(args.checkpoints, setting)
if not os.path.exists(path):
    os.makedirs(path)
               
dump(scaler, open(path + '/Scaler.pkl', 'wb'))
print('[INFO] Scaler saved in file: ', path + '/Scaler.pkl')

# Saving parameters
#
args.save(path)

# Data loaders

In [ ]:
timeenc      = 0 if args.embed != 'timeF' else 1

### Training data

In [ ]:
shuffle_flag = True
drop_last    = True

train_data = createDataset(df        = df_train,
                           size      = [args.seq_len, args.label_len, args.pred_len],
                           features  = args.features,
                           timeenc   = timeenc,
                           freq      = args.freq,
                        )

train_loader = DataLoader(train_data,
                          batch_size  = args.batch_size,
                          shuffle     = shuffle_flag,
                          num_workers = args.num_workers,
                          drop_last   = drop_last)

### Validation data

In [ ]:
shuffle_flag = True
drop_last    = True

valid_data = createDataset(df        = df_valid,
                           size      = [args.seq_len, args.label_len, args.pred_len],
                           features  = args.features,
                           timeenc   = timeenc,
                           freq      = args.freq,
                        )

valid_loader = DataLoader(valid_data,
                          batch_size  = args.batch_size,
                          shuffle     = shuffle_flag,
                          num_workers = args.num_workers,
                          drop_last   = drop_last)

### Testing data

In [ ]:
shuffle_flag = False
drop_last    = False

test_data = createDataset(df          = df_test,
                          size        = [args.seq_len, args.label_len, args.pred_len],
                          features    = args.features,
                          timeenc     = timeenc,
                          freq        = args.freq)

test_loader = DataLoader(test_data,
                          batch_size  = args.batch_size,
                          shuffle     = shuffle_flag,
                          num_workers = args.num_workers,
                          drop_last   = drop_last)

# Forecasting model

## Setup model

In [ ]:
from exp.exp_main import Exp_Main

# Set experiment
#
Exp = Exp_Main
exp = Exp( args )  

In [ ]:
model = exp.train(setting, train_loader, valid_loader, test_loader);

# Evaluation

## Get predictions

In [ ]:
# Get device for inference
#
device = exp.device
 
    
testY, y_pred = None, None
#
#
with tqdm(test_loader, unit="batch") as tepoch:
    for Iter, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(tepoch):
        tepoch.set_description(f"Iteration: {Iter + 1}/{len(test_loader)}")
    
        batch_x = batch_x.float().to( device )

        batch_x_mark = batch_x_mark.float().to( device )
        batch_y_mark = batch_y_mark.float().to( device )

        # Decoder input
        #
        dec_inp = torch.zeros_like(batch_x).float()
        dec_inp = torch.cat([batch_x[:, -args.label_len:, :], dec_inp], dim=1).float().to( device )

        # Encoder - Decoder
        #
        if args.use_amp:
            with torch.cuda.amp.autocast():
                if args.output_attention:
                    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                else:
                    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        else:
            if args.output_attention:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]

            else:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

        f_dim = -1 if args.features == 'MS' else 0


        # Get predictions and real values
        #
        pred    = outputs[:, -args.pred_len:, f_dim:]
        true    = batch_y[:, -args.pred_len:, f_dim:].to( device )

        # Convert to NumPy arrays
        #
        pred = pred.detach().cpu().numpy()
        true = true.detach().cpu().numpy()

        # Inverse trasformation
        #
        pred = scaler.inverse_transform( pred )
        true = scaler.inverse_transform( true )
        if ( args.transformation ):
            pred = np.exp( pred )
            true = np.exp( true )


        # Store predictions & Real values
        #
        if (testY is None):
            y_pred = pred
            testY  = true
        else:
            y_pred = np.concatenate([y_pred, pred], axis = 0)
            testY  = np.concatenate([testY,  true], axis = 0)  

            
         
        
        # Print Examples to pdf
        #
        if (args.saveExamples and Iter % 10 == 0):
            from utils.tools import visual
            
            # Get input
            #
            inputs = batch_x.detach().cpu().numpy()

            # Inverse trasformation
            #
            inputs = scaler.inverse_transform( inputs )
            if ( args.transformation ):
                inputs = np.exp( inputs )

            # Select only target values
            #
            inputs       = inputs[:, :, -1]
            GroundTruth = true[:, :, -1]
            Predictions = pred[:, :, -1]


            # Create directory for model checkpoints
            #
            Images_path = os.path.join(args.checkpoints, setting, 'pics')
            if not os.path.exists(Images_path):
                os.makedirs(Images_path)

            visual(input, GroundTruth, Predictions, figsize = (20, 3), name = os.path.join(Images_path, f'{Iter}.pdf'));    

### Keep only useful features

In [ ]:
testY  = testY[:,:,-1]
y_pred = y_pred[:,:,-1]

## Calculate Performance on Testing set


In [ ]:
Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

filename = os.path.join(args.checkpoints, setting, 'results.txt')

with open(filename, 'w') as f:
    # Legend
    f.write('Horizon\t   MAE   SMAPE    RMSE     R2\n')
    for i in range( args.pred_len ):

        # Create DataFrame
        #
        Prices = pd.DataFrame([])        
        
        # Include Real values and Predictions
        #
        Prices['Real']       = testY[:,  i]
        Prices['Prediction'] = y_pred[:, i] 



        # Evaluation
        #
        MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )

        # Print results
        #
        print('Horizon: %2i  MAE: %.2f  RMSE: %.2f  SMAPE: %.2f  R2: %.2f' % (i+1, MAE, RMSE, SMAPE, R2) )
        f.write('{:7.0f}\t {:.3f}   {:.3f}   {:.3f}  {:.3f}\n'.format(i+1, MAE, SMAPE, RMSE, R2))

    f.close()

## Prediction visualization


In [ ]:
Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

for i in range( 6 ):

    Prices = pd.DataFrame([])        

    Prices['Real']       = testY[:,  i]
    Prices['Prediction'] = y_pred[:, i] 
            
    
    # Plot Real & Predicted values
    #
    Prices[:100].plot( figsize = (20, 3), marker = 'o' )
    #
    plt.title('Feature: {} - Horizon = {}'.format('Target', i+1))
    plt.legend( frameon = False, fontsize = 14)
    plt.xticks(size = 12)
    plt.yticks(size = 12)
    plt.show()        

## Examples

In [ ]:
subplots = [331, 332, 333, 334, 335, 336,  337, 338, 339]
plt.figure( figsize = (30, 10) )
RandomInstances = [random.randint(1,testY.shape[0]) for i in range(0, 9)]

for plot_id, i in enumerate(RandomInstances):
    
    plt.subplot(subplots[plot_id])
    plt.grid()
    plt.plot(range(args.pred_len), testY[i],  color='g', marker = 'o')
    plt.plot(range(args.pred_len), y_pred[i], color='r', marker = 'o')

    plt.legend(['Future values', 'Prediction'], frameon = False, fontsize = 12)
plt.show()